# Thomas Fire Analysis

#### Author: Tom Gibbens-Matsuyama (tommats00@github.io)
- This notebook is part of my Thomas Fire analysis [github repository](https://github.com/tommats00/eds220-final) 

Image: ![Thomas Fire 2017](https://media.gettyimages.com/id/886375862/photo/ventura-county-fire.jpg?s=612x612&w=gi&k=20&c=Te_tMpB-C35EhGdmCqRGdwYvo1kPQ-yVeESXMY1gX7M=)

Image Credits: Marcus Yam

# About

### Purpose
Paragraph


### Highlights of analysis:
- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8

### Dataset descriptions:

#### AQI Data
- 1
- 2

#### Landsat Collection 2 Level-2 data from Landsat 8 
- 1
- 2

#### References & data sources



## Data analysis sections
- break the analyssi into logical sections
- include only the most relevant code
- aim for clean, proessional layout with properly formatted code snippets and images 

### Final visualizations
- Showcase your main visualizations with clear captions. 